In [ ]:
# 🔑 Configure sua API Key da OpenRouter (opcional)
# Se não configurar, o sistema funcionará sem monitoramento LLM

OPENROUTER_API_KEY = ""  # @param {type:"string"}

# Configurar como variável de ambiente se fornecida
import os
if OPENROUTER_API_KEY.strip():
    os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY
    print("✅ API Key OpenRouter configurada - Monitor LLM ativado")
else:
    print("⚠️  API Key não configurada - Monitor LLM desabilitado")
    print("   O treinamento funcionará normalmente sem monitoramento inteligente")


In [ ]:
# Verificar GPU disponível
!nvidia-smi

# Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navegar para diretório de trabalho
%cd /content


In [ ]:
# Clone do repositório ValeTTS atualizado
!git clone https://github.com/wallaceblaia/ValeTTS-Colab.git ValeTTS
%cd ValeTTS

# Verificar estrutura do projeto
print("📁 Estrutura do projeto:")
!ls -la

# Verificar se os arquivos necessários existem
print("\n🔍 Verificando arquivos essenciais:")
!ls -la requirements.txt setup.py 2>/dev/null || echo "❌ Arquivos não encontrados"
print("📁 Diretório atual:", os.getcwd())


In [ ]:
# 📦 INSTALAÇÃO COMPLETA DE DEPENDÊNCIAS (INGLÊS + PORTUGUÊS)
import os
print(f"📁 Diretório atual: {os.getcwd()}")

# Dependências core do projeto
if os.path.exists('requirements.txt'):
    print("📦 Instalando dependências do projeto...")
    !pip install -r requirements.txt
else:
    print("⚠️  requirements.txt não encontrado, instalando dependências básicas...")
    !pip install torch torchaudio pytorch-lightning tensorboard pyyaml matplotlib scipy librosa

# 🌍 DEPENDÊNCIAS ESPECÍFICAS PARA PROCESSAMENTO DE INGLÊS
print("\n🌍 Instalando suporte específico para inglês...")

# Phonemizer para conversão texto-para-phonemes em inglês
print("🔤 Instalando phonemizer...")
!pip install phonemizer --quiet

# NLTK para processamento avançado de texto
print("📝 Instalando NLTK...")
!pip install nltk --quiet

# eSpeak backend para phonemizer (necessário para inglês)
print("🔊 Configurando eSpeak para phonemes ingleses...")
!sudo apt-get update -qq
!sudo apt-get install espeak espeak-data -y -qq

# Lightning para treinamento (versão mais recente)
print("⚡ Atualizando PyTorch Lightning...")
!pip install lightning --quiet

print("\n🧪 Testando phonemizer...")
try:
    from phonemizer import phonemize
    test_phonemes = phonemize("hello world", language='en-us', backend='espeak')
    print(f"✅ Phonemizer funcionando: '{test_phonemes}'")
except Exception as e:
    print(f"⚠️  Erro no phonemizer: {e}")

# Instalar o projeto em modo desenvolvimento
if os.path.exists('setup.py'):
    print("\n🔧 Instalando projeto em modo desenvolvimento...")
    !pip install -e .
else:
    print("\n⚠️  setup.py não encontrado, configurando PYTHONPATH...")
    import sys
    if os.getcwd() not in sys.path:
        sys.path.insert(0, os.getcwd())

print("\n✅ TODAS AS DEPENDÊNCIAS INSTALADAS!")
print("   🔤 Phonemizer: Conversão G2P para inglês")
print("   📝 NLTK: Processamento de texto avançado")
print("   🔊 eSpeak: Backend para phonemes ingleses")
print("   ⚡ Lightning: Treinamento otimizado")


In [ ]:
# Criar diretório de dados
!mkdir -p data/generated

# Extrair dataset do Google Drive
import os
dataset_path = "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz"

if os.path.exists(dataset_path):
    print("📦 Extraindo dataset em inglês...")
    !tar -xzf "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz" -C data/generated/

    # Verificar extração
    print("\n📁 Conteúdo do dataset:")
    !ls -la data/generated/Dataset-Unificado/

    # Verificar metadados e idioma
    print("\n📋 Verificando estrutura do dataset:")
    !head -n 1 data/generated/Dataset-Unificado/metadata.csv

    print("\n🌍 Verificando idioma do dataset:")
    # Verificar coluna locale (posição 9 no CSV)
    !cut -d',' -f9 data/generated/Dataset-Unificado/metadata.csv | head -n 5 | tail -n 4

    print("\n📊 Estatísticas do dataset:")
    total_lines = !wc -l data/generated/Dataset-Unificado/metadata.csv
    print(f"Total de amostras: {total_lines[0].split()[0]} linhas")

    # Verificar algumas amostras de texto
    print("\n📝 Amostras de texto (verificando inglês):")
    !cut -d',' -f18 data/generated/Dataset-Unificado/metadata.csv | head -n 4 | tail -n 3

    print("\n✅ Dataset em inglês restaurado com sucesso!")
else:
    print(f"❌ Dataset não encontrado em: {dataset_path}")
    print("   Verifique se o arquivo existe no Google Drive")


In [ ]:
# Verificação completa do sistema
import os

print("🔍 Verificação completa do sistema:")
print(f"📁 Diretório atual: {os.getcwd()}")

# Verificar estrutura essencial
essential_files = [
    'scripts/train_vits2.py',
    'configs/training/vits2_dataset_unificado.yaml',
    'valetts/__init__.py'
]

print("\n📋 Verificando arquivos essenciais:")
for file in essential_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - FALTANDO")

# Verificar se o módulo valetts pode ser importado
try:
    import valetts
    print("✅ Módulo valetts importado com sucesso")
except ImportError as e:
    print(f"⚠️  Erro ao importar valetts: {e}")
    print("   Adicionando diretório ao PYTHONPATH...")
    import sys
    if os.getcwd() not in sys.path:
        sys.path.insert(0, os.getcwd())
    try:
        import valetts
        print("✅ Módulo valetts importado após ajuste do PATH")
    except ImportError as e2:
        print(f"❌ Ainda não conseguiu importar: {e2}")

# Testar script de treinamento
if os.path.exists('scripts/train_vits2.py'):
    print("\n🧪 Testando sistema de treinamento...")
    !python scripts/train_vits2.py --help
    print("\n✅ Sistema funcionando corretamente!")
else:
    print("\n❌ Script de treinamento não encontrado!")
    print("   Listando conteúdo do diretório scripts:")
    !ls -la scripts/ 2>/dev/null || echo "Diretório scripts não encontrado"


In [ ]:
# Verificar e mostrar configuração que será usada (INGLÊS)
import yaml
import os

config_file = 'configs/training/vits2_english_dataset.yaml'

# Verificar se arquivo existe
if not os.path.exists(config_file):
    print(f"❌ Arquivo de configuração não encontrado: {config_file}")
    print("📁 Arquivos disponíveis em configs/training/:")
    !ls -la configs/training/ 2>/dev/null || echo "Diretório não encontrado"
else:
    # Carregar configuração
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)

    print("📋 Configuração do Treinamento:")

    # Função auxiliar para acessar chaves de forma segura
    def safe_get(dictionary, keys, default="N/A"):
        try:
            for key in keys:
                dictionary = dictionary[key]
            return dictionary
        except (KeyError, TypeError):
            return default

    # Mostrar configurações principais
    print(f"   • Dataset Dir: {safe_get(config, ['data', 'data_dir'])}")
    print(f"   • Metadata: {safe_get(config, ['data', 'metadata_file'])}")
    print(f"   • Speakers: {safe_get(config, ['model', 'n_speakers'])}")
    print(f"   • Epochs: {safe_get(config, ['training', 'max_epochs'])}")
    print(f"   • Batch Size: {safe_get(config, ['training', 'batch_size'])}")
    print(f"   • Learning Rate: {safe_get(config, ['training', 'learning_rate'])}")
    print(f"   • Mixed Precision: {safe_get(config, ['training', 'use_amp'])}")
    print(f"   • Sample Rate: {safe_get(config, ['data', 'sample_rate'])}")
    print(f"   • Mel Channels: {safe_get(config, ['data', 'n_mels'])}")

    # Monitor LLM
    if 'llm_monitor' in config and config['llm_monitor'].get('enabled', False):
        print(f"   • Monitor LLM: ✅ Ativado")
        print(f"     - Modelo: {safe_get(config, ['llm_monitor', 'model'])}")
        print(f"     - Monitor a cada: {safe_get(config, ['llm_monitor', 'monitor_every_epochs'])} épocas")
        print(f"     - Provider: {safe_get(config, ['llm_monitor', 'provider'])}")
    else:
        print(f"   • Monitor LLM: ❌ Desabilitado")

    print("\n🎯 Configuração carregada com sucesso!")


In [ ]:
# 🎯 COMANDO EXATO DO AMBIENTE LOCAL (INGLÊS)
# python scripts/train_vits2.py --config configs/training/vits2_english_dataset.yaml

import os
import sys

# Garantir que o módulo valetts esteja no PYTHONPATH
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
    print(f"📁 Adicionado ao PYTHONPATH: {current_dir}")

# Verificar se o módulo valetts pode ser importado
try:
    import valetts
    print("✅ Módulo valetts importado com sucesso")
    print(f"   Localização: {valetts.__file__}")
except ImportError as e:
    print(f"❌ Erro ao importar valetts: {e}")
    print("🔧 Tentando ajustar imports...")

    # Verificar estrutura
    print("\n📋 Verificando estrutura:")
    !ls -la valetts/ 2>/dev/null || echo "❌ Diretório valetts não encontrado"
    !ls -la valetts/models/ 2>/dev/null || echo "❌ Diretório valetts/models não encontrado"

    # Tentar importar novamente
    try:
        import valetts
        print("✅ Módulo valetts importado após ajustes")
    except ImportError as e2:
        print(f"❌ Ainda não conseguiu importar: {e2}")
        print("⚠️  Continuando com instalação manual...")

# Determinar se deve desabilitar LLM baseado na API key
disable_llm = "" if os.environ.get('OPENROUTER_API_KEY') else "--disable-llm"

print("\n" + "="*50)
print("🚀 Iniciando treinamento VITS2 para INGLÊS...")
print(f"📁 Configuração: configs/training/vits2_english_dataset.yaml")
print(f"🌍 Dataset: Dataset-Unificado (inglês, 22.9K amostras)")
print(f"🔧 Monitor LLM: {'Ativado' if not disable_llm else 'Desabilitado'}")
print(f"🎤 Geração de amostras: Ativada (textos em inglês)")
print(f"🐍 Python path: {current_dir}")
print("="*50)

# Executar treinamento com PYTHONPATH explícito
os.environ['PYTHONPATH'] = current_dir
!PYTHONPATH={current_dir} python scripts/train_vits2.py --config configs/training/vits2_english_dataset.yaml {disable_llm}


In [ ]:
# 🔧 BACKUP: Instalação manual do módulo valetts (se necessário)

import subprocess
import sys
import os

def install_valetts_module():
    """Instala o módulo valetts em modo de desenvolvimento"""
    try:
        # Tentar importar primeiro
        import valetts
        print("✅ Módulo valetts já está funcionando!")
        return True
    except ImportError:
        print("⚠️  Módulo valetts não encontrado. Instalando...")

        try:
            # Instalar em modo de desenvolvimento
            result = subprocess.run(
                [sys.executable, "-m", "pip", "install", "-e", "."],
                cwd="/content/ValeTTS",
                capture_output=True,
                text=True
            )

            if result.returncode == 0:
                print("✅ Módulo valetts instalado com sucesso!")

                # Verificar instalação
                import valetts
                print(f"   Localização: {valetts.__file__}")
                return True
            else:
                print(f"❌ Erro na instalação: {result.stderr}")
                return False

        except Exception as e:
            print(f"❌ Erro durante instalação: {e}")
            return False

# Executar verificação/instalação
if not install_valetts_module():
    print("\n🚨 AVISO: Não foi possível instalar o módulo valetts automaticamente.")
    print("   Você pode tentar executar manualmente:")
    print("   !pip install -e .")
    print("   ou verificar se todos os arquivos foram extraídos corretamente.")


In [ ]:
# Carregar TensorBoard para monitoramento
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard/ --port 6006


In [ ]:
# Verificar progresso do treinamento
print("📊 Status do Treinamento:")
!ls -la logs/

print("\n📁 Checkpoints:")
!find logs/ -name "*.ckpt" | tail -5

print("\n📈 Logs mais recentes:")
!tail -10 logs/training.log 2>/dev/null || echo "Log não encontrado ainda"


In [ ]:
# Backup de modelos para Google Drive
import os
from shutil import copytree, ignore_patterns

backup_dir = "/content/drive/MyDrive/ValeTTS-Colab/training_backup"

if os.path.exists("logs/"):
    print("💾 Fazendo backup dos logs e checkpoints...")
    !mkdir -p "{backup_dir}"
    !cp -r logs/ "{backup_dir}/" 2>/dev/null || echo "Backup parcial realizado"
    print(f"✅ Backup salvo em: {backup_dir}")
else:
    print("ℹ️  Nenhum log encontrado para backup")


In [ ]:
# 🎵 EXPLORAR AMOSTRAS DE ÁUDIO GERADAS DURANTE O TREINAMENTO

import os
import glob
import json
from IPython.display import Audio, display, HTML
from pathlib import Path

def explore_generated_samples():
    """Explora e lista amostras de áudio geradas durante o treinamento."""

    samples_dir = Path("samples")

    if not samples_dir.exists():
        print("⚠️  Diretório 'samples' não encontrado.")
        print("   As amostras são geradas automaticamente durante o treinamento.")
        print("   Execute o treinamento primeiro para ver amostras aqui.")
        return

    # Listar épocas disponíveis
    epoch_dirs = sorted([d for d in samples_dir.iterdir() if d.is_dir() and d.name.startswith("epoch_")])

    if not epoch_dirs:
        print("📂 Diretório samples existe, mas não há épocas com amostras ainda.")
        print("   As amostras serão geradas conforme o treinamento progride.")
        return

    print(f"🎵 Encontradas amostras de {len(epoch_dirs)} épocas!")
    print("\n📁 Épocas disponíveis:")

    for epoch_dir in epoch_dirs[-5:]:  # Mostrar últimas 5 épocas
        epoch_num = epoch_dir.name.split("_")[1]
        audio_files = list(epoch_dir.glob("*.wav"))
        print(f"   📊 Época {epoch_num}: {len(audio_files)} amostras")

    # Mostrar detalhes da época mais recente
    if epoch_dirs:
        latest_epoch = epoch_dirs[-1]
        epoch_num = latest_epoch.name.split("_")[1]

        print(f"\n🎯 ÉPOCA MAIS RECENTE: {epoch_num}")
        print(f"📁 Diretório: {latest_epoch}")

        # Verificar resumo da época
        summary_file = latest_epoch / "samples_summary.json"
        if summary_file.exists():
            with open(summary_file, 'r') as f:
                summary = json.load(f)

            print(f"📋 Resumo:")
            print(f"   🕐 Timestamp: {summary.get('timestamp', 'N/A')}")
            print(f"   🎤 Total de amostras: {summary.get('total_samples', 0)}")
            print(f"   🌍 Idioma: {summary.get('language', 'N/A')}")
            print(f"   👥 Speakers usados: {summary.get('speakers_used', 0)}")

        # Listar arquivos de áudio
        audio_files = sorted(latest_epoch.glob("*.wav"))
        if audio_files:
            print(f"\n🎶 Arquivos de áudio ({len(audio_files)}):")
            for audio_file in audio_files:
                print(f"   🔊 {audio_file.name}")

        return latest_epoch, audio_files

    return None, []

# Executar exploração
latest_epoch, audio_files = explore_generated_samples()


In [ ]:
# 🎧 REPRODUZIR AMOSTRAS DE ÁUDIO NO COLAB

def play_audio_samples(epoch_dir=None, max_samples=5):
    """Reproduz amostras de áudio no Colab com interface interativa."""

    if epoch_dir is None:
        # Usar época mais recente
        samples_dir = Path("samples")
        if not samples_dir.exists():
            print("⚠️  Nenhuma amostra encontrada. Execute o treinamento primeiro.")
            return

        epoch_dirs = sorted([d for d in samples_dir.iterdir() if d.is_dir() and d.name.startswith("epoch_")])
        if not epoch_dirs:
            print("⚠️  Nenhuma época com amostras encontrada.")
            return

        epoch_dir = epoch_dirs[-1]

    epoch_num = epoch_dir.name.split("_")[1]
    print(f"🎵 REPRODUZINDO AMOSTRAS DA ÉPOCA {epoch_num}")
    print("=" * 50)

    # Buscar arquivos de áudio
    audio_files = sorted(epoch_dir.glob("*.wav"))

    if not audio_files:
        print("⚠️  Nenhum arquivo de áudio encontrado nesta época.")
        return

    # Carregar resumo se disponível
    summary_file = epoch_dir / "samples_summary.json"
    sample_metadata = {}

    if summary_file.exists():
        with open(summary_file, 'r') as f:
            summary = json.load(f)

        # Criar dicionário de metadados por arquivo
        for sample in summary.get('samples', []):
            audio_path = Path(sample['audio_path'])
            sample_metadata[audio_path.name] = sample

    # Reproduzir amostras
    played_count = 0
    for audio_file in audio_files[:max_samples]:
        if played_count >= max_samples:
            break

        print(f"\n🔊 {audio_file.name}")

        # Mostrar metadados se disponíveis
        if audio_file.name in sample_metadata:
            metadata = sample_metadata[audio_file.name]
            print(f"   📝 Texto: {metadata['text']}")
            print(f"   🎤 Speaker: {metadata['speaker_name']} ({metadata['speaker_id']})")
            if 'metadata' in metadata and 'audio_length_seconds' in metadata['metadata']:
                duration = metadata['metadata']['audio_length_seconds']
                print(f"   ⏱️  Duração: {duration:.2f}s")

        # Reproduzir áudio
        try:
            audio_widget = Audio(str(audio_file), autoplay=False)
            display(audio_widget)
            played_count += 1

        except Exception as e:
            print(f"   ❌ Erro ao carregar áudio: {e}")

    if len(audio_files) > max_samples:
        remaining = len(audio_files) - max_samples
        print(f"\n📊 Mostrando {max_samples} de {len(audio_files)} amostras.")
        print(f"   (+{remaining} amostras adicionais disponíveis)")

    print(f"\n✅ Reprodução concluída! {played_count} amostras carregadas.")

# Reproduzir amostras se disponíveis
if 'latest_epoch' in locals() and latest_epoch:
    play_audio_samples(latest_epoch, max_samples=3)
else:
    print("🎯 Execute a célula anterior primeiro para encontrar amostras disponíveis.")
    print("   Ou rode esta célula após o treinamento ter gerado algumas amostras.")


In [ ]:
# 💾 BACKUP DE AMOSTRAS DE ÁUDIO PARA GOOGLE DRIVE

import shutil
from pathlib import Path

def backup_audio_samples_to_drive():
    """Faz backup das amostras geradas para o Google Drive."""

    samples_dir = Path("samples")
    backup_base = Path("/content/drive/MyDrive/ValeTTS-Colab")
    backup_samples_dir = backup_base / "audio_samples"

    if not samples_dir.exists():
        print("⚠️  Diretório 'samples' não encontrado.")
        print("   Execute o treinamento primeiro para gerar amostras.")
        return False

    # Criar diretório de backup
    backup_samples_dir.mkdir(parents=True, exist_ok=True)

    # Contar épocas e amostras
    epoch_dirs = sorted([d for d in samples_dir.iterdir() if d.is_dir() and d.name.startswith("epoch_")])

    if not epoch_dirs:
        print("📂 Diretório samples existe, mas não há épocas para fazer backup.")
        return False

    print(f"💾 Fazendo backup de {len(epoch_dirs)} épocas para Google Drive...")
    print(f"📁 Destino: {backup_samples_dir}")

    total_files = 0
    total_size = 0

    for epoch_dir in epoch_dirs:
        epoch_backup_dir = backup_samples_dir / epoch_dir.name

        try:
            # Copiar época inteira
            if epoch_backup_dir.exists():
                shutil.rmtree(epoch_backup_dir)

            shutil.copytree(epoch_dir, epoch_backup_dir)

            # Contar arquivos
            audio_files = list(epoch_backup_dir.glob("*.wav"))
            epoch_size = sum(f.stat().st_size for f in epoch_backup_dir.rglob("*") if f.is_file())

            total_files += len(audio_files)
            total_size += epoch_size

            print(f"   ✅ {epoch_dir.name}: {len(audio_files)} amostras ({epoch_size/1024/1024:.1f} MB)")

        except Exception as e:
            print(f"   ❌ Erro no backup de {epoch_dir.name}: {e}")

    print(f"\n📊 BACKUP CONCLUÍDO:")
    print(f"   🎵 Total de amostras: {total_files}")
    print(f"   💾 Tamanho total: {total_size/1024/1024:.1f} MB")
    print(f"   📁 Salvo em: {backup_samples_dir}")

    # Criar arquivo de índice
    index_file = backup_samples_dir / "index.txt"
    with open(index_file, 'w') as f:
        f.write(f"ValeTTS Audio Samples Backup\\n")
        f.write(f"Generated: {Path.cwd()}\\n")
        f.write(f"Total epochs: {len(epoch_dirs)}\\n")
        f.write(f"Total samples: {total_files}\\n")
        f.write(f"Total size: {total_size/1024/1024:.1f} MB\\n\\n")

        for epoch_dir in epoch_dirs:
            epoch_backup_dir = backup_samples_dir / epoch_dir.name
            if epoch_backup_dir.exists():
                audio_files = list(epoch_backup_dir.glob("*.wav"))
                f.write(f"{epoch_dir.name}: {len(audio_files)} samples\\n")

    print(f"   📋 Índice salvo: {index_file}")

    return True

# Executar backup se há amostras disponíveis
if Path("samples").exists():
    backup_success = backup_audio_samples_to_drive()

    if backup_success:
        print("\\n🎉 BACKUP CONCLUÍDO COM SUCESSO!")
        print("   Suas amostras de áudio estão seguras no Google Drive.")
        print("   Você pode baixá-las ou acessá-las diretamente do Drive.")
    else:
        print("\\n⚠️  Backup não foi necessário ou falhou.")
else:
    print("📝 Backup será executado automaticamente quando houver amostras disponíveis.")
    print("   Execute o treinamento primeiro para gerar amostras de áudio.")
